# Process Semi-Structured Data from DataFrames

1. Create Snowpark Dataframes
    
    From Files(JSON, XML)

2. Transform semi-structured data in DataFrames

    Traverse Semi-structures Data

    Explicitly Cast Values in Semi-Structured data

    Flatten an array of Objects into rows

    Load Semi-Structured Data into DataFrames


3. Clean and Enrich data using Snowpark for Python

    Perform Joins


For more information follow the below links:

1. [Working with Semi-Structured Data](https://docs.snowflake.com/en/developer-guide/snowpark/python/working-with-dataframes#working-with-semi-structured-data)

2. [Querying Semi-structured Data](https://docs.snowflake.com/en/user-guide/querying-semistructured)

In [ ]:
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.types import *
from snowflake.snowpark.functions import *

session = get_active_session()

In [ ]:
use Database Snowpark_db;
use schema works;

In [ ]:
JsonData =  session.read.json('@snowpark_db.sourced.stage1/car-sales.ndjson')
JsonData

In [ ]:
#Accessing data using snowpark
df = JsonData.select(sql_expr("$1:customer[0].name::string"))
df

In [ ]:
Use Snowpark_db.sourced;

SELECT src:dealership::string,
    src:salesperson.name::string,
    src['salesperson']['name'],
    src:customer[0].name,
    src:vehicle[0],
    src:vehicle[0].price::int,
    GET(src:vehicle[0], 'make'),
    GET_PATH(src:vehicle, '[0]')
FROM car_sales
ORDER BY 1;

In [ ]:
dft =  session.table("snowpark_db.sourced.car_sales")

df = dft.select(
    col("src")["dealership"].cast(StringType()),
    col("src")["salesperson"]["name"].cast(StringType()),
    col("src")['salesperson']['name'],
    col("src")["customer"][0]["name"],
    sql_expr("src:customer[0].name::string"),
    col("src")["vehicle"][0],
    col("src")["vehicle"][0]["price"].cast(IntegerType()),
    get(col("src")["vehicle"][0], lit("make")),
    get_path(col("src"), lit("vehicle[0]"))
)
df

In [ ]:
SELECT *
FROM car_sales, LATERAL FLATTEN(src:vehicle[0]);

In [ ]:
df = dft.join_table_function("flatten", col("src")["vehicle"]
    ).select(col("value"), col("value")["make"], col("value")["extras"])
df

In [ ]:
SELECT v.value as vehicle, e.value as extras
FROM car_sales,
    LATERAL FLATTEN(src:vehicle) v,
    LATERAL FLATTEN(v.value:extras) e;

In [ ]:
df = dft.join_table_function("flatten", col("src")["vehicle"]
    ).select(col("value").as_("vehicle"))
df = df.join_table_function("flatten", col("vehicle")["extras"]
    ).select(col("vehicle"), col("value").as_("extras"))
df